In [ ]:
import pandas as pd

import matplotlib.pyplot as plt
%matplotlib inline

from sklearn.linear_model import LogisticRegression
from sklearn.neural_network import MLPClassifier
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier

from sklearn.metrics import ConfusionMatrixDisplay, precision_score, recall_score, f1_score

from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

In [ ]:
pd.options.display.max_columns = 100

### Reading the dataset

In [ ]:
music_df = pd.read_csv('./data/msd_genre_dataset.txt')

In [ ]:
music_df.shape

In [ ]:
music_df.dtypes

In [ ]:
music_df.head()

In [ ]:
music_df['genre'].value_counts(normalize = True)

In [ ]:
genres = ['jazz and blues', 'soul and reggae']
music_df_filter = music_df.loc[music_df['genre'].isin(genres)]

In [ ]:
music_df_filter.shape

In [ ]:
music_df_filter.loc[music_df_filter['genre'] == 'jazz and blues', 'genre'] = 0
music_df_filter.loc[music_df_filter['genre'] == 'soul and reggae', 'genre'] = 1

In [ ]:
music_df_filter['genre'].value_counts(normalize = True)

### Splitting train and test datasets

In [ ]:
X = music_df_filter[music_df_filter.columns.tolist()[4:]]

In [ ]:
Y = music_df_filter['genre']

In [ ]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size = 0.3, stratify = Y)

In [ ]:
pd.Series(Y_train).value_counts(normalize = True)

In [ ]:
pd.Series(Y_test).value_counts(normalize = True)

### Preprocessing feature matrix

In [ ]:
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)

In [ ]:
X_test = scaler.transform(X_test)

### Training a Logistic Regression model

In [ ]:
logmodel = LogisticRegression()

In [ ]:
logmodel.fit(X_train, Y_train.astype(int))

In [ ]:
logmodel_predictions = logmodel.predict(X_test)

In [ ]:
ConfusionMatrixDisplay.from_predictions(Y_test.astype(int), logmodel_predictions)

In [ ]:
print('Precision:', precision_score(Y_test.astype(int), logmodel_predictions))
print('Recall:', recall_score(Y_test.astype(int), logmodel_predictions))
print('F1:', f1_score(Y_test.astype(int), logmodel_predictions))

### Training a Neural Networks model

In [ ]:
neuralmodel = MLPClassifier()

In [ ]:
neuralmodel.fit(X_train, Y_train.astype(int))

In [ ]:
neuralmodel_predictions = neuralmodel.predict(X_test)

In [ ]:
ConfusionMatrixDisplay.from_predictions(Y_test.astype(int), neuralmodel_predictions)

In [ ]:
print('Precision:', precision_score(Y_test.astype(int), neuralmodel_predictions))
print('Recall:', recall_score(Y_test.astype(int), neuralmodel_predictions))
print('F1:', f1_score(Y_test.astype(int), neuralmodel_predictions))

### Training a Support Vector Machines model

In [ ]:
svmmodel = SVC()

In [ ]:
svmmodel.fit(X_train,Y_train.astype(int))

In [ ]:
svmmodel_predictions = svmmodel.predict(X_test)

In [ ]:
ConfusionMatrixDisplay.from_predictions(Y_test.astype(int), svmmodel_predictions)

In [ ]:
print('Precision:', precision_score(Y_test.astype(int), svmmodel_predictions))
print('Recall:', recall_score(Y_test.astype(int), svmmodel_predictions))
print('F1:', f1_score(Y_test.astype(int), svmmodel_predictions))

### Training a Random Forest model